# de base de datos a json

In [205]:

from fastapi import FastAPI, requests, HTTPException
import uvicorn
import os
from dotenv import load_dotenv
import psycopg2
import json
import pandas as pd
load_dotenv()
    
#Leer Las Variables:
host = os.getenv("DB_HOST_AWS")
username = os.getenv("DB_USER_AWS")
password = os.getenv("DB_PASSWORD_AWS")
database = os.getenv("DB_DATABASE_AWS")
port = os.getenv("DB_PORT_AWS")
try:
    conn = psycopg2.connect(host=host,
                                database=database,
                                user=username,
                                password=password,
                                port=port,
                                sslmode="require")
    
    print("Conexión exitosa a la base de datos PostgreSQL con SSL")
except psycopg2.OperationalError as e:
    raise HTTPException(status_code=400, detail="Error de conexión: " + str(e))
except Exception as error:
    raise HTTPException(status_code=400, detail="Error desconocido: " + str(e))
cur = conn.cursor()


Conexión exitosa a la base de datos PostgreSQL con SSL


In [206]:
# Escribe la consulta SQL
query = """SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public' AND table_type = 'BASE TABLE';
"""
# query = "SELECT * FROM preguntas_front"

# Usa pandas para ejecutar la consulta y convertirla en un DataFrame
df = pd.read_sql_query(query, conn)
df.head(50)

C:\Users\2012f\AppData\Local\Temp\ipykernel_8440\4123380349.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,table_name
0,preguntas_opciones_chatbot
1,no_sociosanit_formulario
2,opciones_chatbot
3,admin_data
4,categorias_chatbot
5,preguntas
6,sociosanitarios_formulario
7,preguntas_chatbot
8,preguntas_front
9,categoria_pregunta_chat_intermed


In [207]:
# Escribe la consulta SQL
query = """
        SELECT * FROM preguntas_opciones_chatbot
        """
df = pd.read_sql_query(query, conn)
df.head(50)

C:\Users\2012f\AppData\Local\Temp\ipykernel_8440\3171491949.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,id_pregunta,id_opcion


In [208]:
# Escribe la consulta SQL
query = """
        SELECT * FROM opciones_chatbot
        """
df = pd.read_sql_query(query, conn)
df.head(50)

C:\Users\2012f\AppData\Local\Temp\ipykernel_8440\128316585.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,id_opcion,texto_opcion
0,1,Manejo clínico de pacientes con vih
1,2,Protocolo PEP
2,3,"Tratamientos (PREP, TAR)"
3,4,Prevención de infecciones oportunistas
4,5,Consejería para adherencia al tratamiento
5,6,Acceso a medicamentos o servicios
6,7,Recursos legales y derechos
7,8,Apoyo a personas en situación de vulnerabilidad
8,9,Conexión con grupos de apoyo comunitario
9,10,Información sobre redes de Servicios Sociales


# Cargar datos

In [199]:
# Recuperar los id_pregunta de la tabla preguntas_chatbot
cur.execute("SELECT id_pregunta, texto_pregunta FROM preguntas_chatbot")
preguntas_dict = cur.fetchall()  # Esto es una lista de tuplas con (id_pregunta, pregunta)

# Recuperar los id_opcion de la tabla opciones_chatbot
cur.execute("SELECT id_opcion, texto_opcion FROM opciones_chatbot")
opciones_dict = cur.fetchall()  # Esto es una lista de tuplas con (id_opcion, opcion)


In [200]:
# Crear mapeo de preguntas a ids
preguntas_dict = {pregunta[1]: pregunta[0] for pregunta in preguntas_dict}

# Crear mapeo de opciones a ids
opciones_dict = {opcion[1]: opcion[0] for opcion in opciones_dict}

print(preguntas_dict)
# print(opciones_dict)


{'¿Qué apoyo necesitas?': 1, '¿Qué necesitas como Trabajador Social?': 2, '¿Cuándo ocurrió la posible infección?': 3, '¿Necesitas recursos de referencia?': 4, '¿Tienes acceso a un médico?': 5, '¿Has compartido tu preocupación con alguien?': 6, '¿Qué necesitas como Psicologo?': 7, '¿Ha sido en un entorno de "chem-sex"?': 8, '¿Estás en tratamiento TAR?': 9, '¿Has compartido tu diagnostico con alguien?': 10, '¿Tiene acceso a recursos locales o grupos de apoyo?': 11, '¿Qué necesitas como Voluntario/Cuidador?': 12, '¿Cuándo te diagnosticaron?': 13, '¿Has compartido su preocupación sobre esta persona con alguien?': 14, '¿Sabes qué es la PEP?': 15, '¿Qué necesitas como Personal Sanitario?': 16, '¿Qué tipo de exposición fue?': 17, '¿Qué necesitas como Educador?': 18, '¿Quieres información sobre algun tema?': 19}


In [201]:
print(opciones_dict)

{'Manejo clínico de pacientes con vih': 1, 'Protocolo PEP': 2, 'Tratamientos (PREP, TAR)': 3, 'Prevención de infecciones oportunistas': 4, 'Consejería para adherencia al tratamiento': 5, 'Acceso a medicamentos o servicios': 6, 'Recursos legales y derechos': 7, 'Apoyo a personas en situación de vulnerabilidad': 8, 'Conexión con grupos de apoyo comunitario': 9, 'Información sobre redes de Servicios Sociales': 10, 'Apoyo emocional para personas recién diagnosticadas': 11, 'Intervenciones para adherencia al tratamiento': 12, 'Manejo del estigma y problemas de salud mental': 13, 'Recursos para pacientes con vih y trastornos psicológicos': 14, 'Consejería en prevención y autocuidado': 15, 'Material educativo sobre vih': 16, 'Capacitación en prevención': 17, 'Métodos para combatir el estigma': 18, 'Recursos para sensibilización': 19, 'Estadísticas y datos actualizados': 20, 'Info básica sobre vih': 21, 'Consejos para apoyar emocionalmente': 22, 'Recursos legales y sociales para pacientes': 23

In [202]:
print("Preguntas dict:", preguntas_dict)
print("Opciones dict:", opciones_dict)


Preguntas dict: {'¿Qué apoyo necesitas?': 1, '¿Qué necesitas como Trabajador Social?': 2, '¿Cuándo ocurrió la posible infección?': 3, '¿Necesitas recursos de referencia?': 4, '¿Tienes acceso a un médico?': 5, '¿Has compartido tu preocupación con alguien?': 6, '¿Qué necesitas como Psicologo?': 7, '¿Ha sido en un entorno de "chem-sex"?': 8, '¿Estás en tratamiento TAR?': 9, '¿Has compartido tu diagnostico con alguien?': 10, '¿Tiene acceso a recursos locales o grupos de apoyo?': 11, '¿Qué necesitas como Voluntario/Cuidador?': 12, '¿Cuándo te diagnosticaron?': 13, '¿Has compartido su preocupación sobre esta persona con alguien?': 14, '¿Sabes qué es la PEP?': 15, '¿Qué necesitas como Personal Sanitario?': 16, '¿Qué tipo de exposición fue?': 17, '¿Qué necesitas como Educador?': 18, '¿Quieres información sobre algun tema?': 19}
Opciones dict: {'Manejo clínico de pacientes con vih': 1, 'Protocolo PEP': 2, 'Tratamientos (PREP, TAR)': 3, 'Prevención de infecciones oportunistas': 4, 'Consejería pa

In [ ]:
# Relación de preguntas con opciones
preguntas_opciones = [
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Manejo clínico de pacientes con vih']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Protocolo PEP']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Tratamientos (PREP, TAR)']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Prevención de infecciones oportunistas']),
    (preguntas_dict['¿Qué necesitas como Personal Sanitario?'], opciones_dict['Consejería para adherencia al tratamiento']),

]

    
for pregunta_text, opcion_text in preguntas_opciones:
    if pregunta_text in preguntas_dict and opcion_text in opciones_dict:
        pregunta_id = preguntas_dict[pregunta_text]
        opcion_id = opciones_dict[opcion_text]
        cur.execute('''
            INSERT INTO preguntas_opciones_chatbot (pregunta_id, opcion_id)
            VALUES (?, ?)
        ''', (pregunta_id, opcion_id))
    else:
        print(f"Clave no encontrada: Pregunta - {pregunta_text}, Opción - {opcion_text}")

KeyError: '¿Qué necesitas como Personal Sanitario?'

In [120]:
cur = conn.cursor()

# Query para eliminar la tabla
query = """CREATE TABLE IF NOT EXISTS preguntas_opciones_chatbot (
    id_pregunta INT,
    id_opcion INT,
    PRIMARY KEY (id_pregunta, id_opcion),
    FOREIGN KEY (id_pregunta) REFERENCES preguntas_chatbot(id_pregunta) ON DELETE CASCADE,
    FOREIGN KEY (id_opcion) REFERENCES opciones_chatbot(id_opcion) ON DELETE CASCADE
);
"""

# Ejecutar la eliminación
cur.execute(query)
conn.commit()

# JUNTAR EN UN TABLA CATEGORIAS Y OPCIONES

In [51]:
# Escribe la consulta SQL
query = """
        SELECT 
    cpc.id_categoria, 
    cpc.id_pregunta,
    cat.titulo_categoria,
    preg.texto_pregunta
FROM 
    categoria_pregunta_chat_intermed cpc
JOIN 
    categorias_chatbot cat ON cpc.id_categoria = cat.id_categoria
JOIN 
    preguntas_chatbot preg ON cpc.id_pregunta = preg.id_pregunta;

        """
df = pd.read_sql_query(query, conn)
df.head(50)

C:\Users\2012f\AppData\Local\Temp\ipykernel_8440\3239149047.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,id_categoria,id_pregunta,titulo_categoria,texto_pregunta
0,1,16,Personal Sanitario,¿Qué necesitas como Personal Sanitario?
1,2,2,Trabajador Social,¿Qué necesitas como Trabajador Social?
2,3,7,Psicologo,¿Qué necesitas como Psicologo?
3,4,18,Educador,¿Qué necesitas como Educador?
4,5,12,Voluntarios y Cuidadores,¿Qué necesitas como Voluntario/Cuidador?
5,6,13,Tengo vih,¿Cuándo te diagnosticaron?
6,6,9,Tengo vih,¿Estás en tratamiento TAR?
7,6,5,Tengo vih,¿Tienes acceso a un médico?
8,6,10,Tengo vih,¿Has compartido tu diagnostico con alguien?
9,6,11,Tengo vih,¿Tiene acceso a recursos locales o grupos de a...


# OTRO

In [ ]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)

In [12]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)


{
    "1.1": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace menos de 6 meses",
                "Entre 6 meses y un año",
                "Hace más de un año"
            ],
            "¿Estás en tratamiento TAR?": [
                "Sí",
                "No",
                "No estoy seguro"
            ],
            "¿Tienes acceso a un médico?": [
                "Sí",
                "No"
            ]
        }
    },
    "1.2": {
        "titulo": "Creo que me he expuesto al virus",
        "preguntas": {
            "¿Cuándo ocurrió la posible infección?": [
                "Últimas 72h",
                "Hace más de 72h"
            ]
        }
    },
    "2.1": {
        "titulo": "Personal sanitario",
        "preguntas": {
            "¿que necesitas?": [
                "Manejo clínico de pacientes con VIH",
                "Protocolo PEP"
            ]
        }
    },
    "2.2": {
        "titulo

In [12]:
import pandas as pd

# Tabla de categorías
categorias = pd.DataFrame({
    "id_categoria": [1, 2, 3],
    "titulo_categoria": ["Tengo VIH", "Creo que me he expuesto al virus", "Personal sanitario"]
})

# Tabla de preguntas
preguntas = pd.DataFrame({
    "id_pregunta": [1, 2, 3, 4, 5],
    "id_categoria": [1, 1, 1, 2, 3],
    "texto_pregunta": [
        "¿Cuándo te diagnosticaron?",
        "¿Estás en tratamiento TAR?",
        "¿Tienes acceso a un médico?",
        "¿Cuándo ocurrió la posible infección?",
        "¿Qué necesitas?"
    ]
})

# Tabla de opciones
opciones = pd.DataFrame({
    "id_opcion": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "id_pregunta": [1, 1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5],
    "texto_opcion": [
        "Hace menos de 6 meses",
        "Entre 6 meses y un año",
        "Hace más de un año",
        "Sí",
        "No",
        "No estoy seguro",
        "Sí",
        "No",
        "Últimas 72h",
        "Hace más de 72h",
        "Manejo clínico de pacientes con VIH",
        "Protocolo PEP"
    ]
})



In [13]:

# 1. Unir preguntas con categorías
preguntas_categorias = pd.merge(
    preguntas,
    categorias,
    on="id_categoria",
    how="inner"
)

# 2. Unir con opciones
final_table = pd.merge(
    opciones,
    preguntas_categorias,
    on="id_pregunta",
    how="inner"
)

In [16]:
# Lista de columnas que deben ir primero
columns_first = ["id_categoria", "id_pregunta", "id_opcion"]

# Añade las columnas restantes
columns_rest = [col for col in final_table.columns if col not in columns_first]

# Reordena las columnas
final_table = final_table[columns_first + columns_rest]

print(final_table)


    id_categoria  id_pregunta  id_opcion                         texto_opcion  \
0              1            1          1                Hace menos de 6 meses   
1              1            1          2               Entre 6 meses y un año   
2              1            1          3                   Hace más de un año   
3              1            2          4                                   Sí   
4              1            2          5                                   No   
5              1            2          6                      No estoy seguro   
6              1            3          7                                   Sí   
7              1            3          8                                   No   
8              2            4          9                          Últimas 72h   
9              2            4         10                      Hace más de 72h   
10             3            5         11  Manejo clínico de pacientes con VIH   
11             3            

In [17]:
final_table

,id_categoria,id_pregunta,id_opcion,texto_opcion,texto_pregunta,titulo_categoria
0,1,1,1,Hace menos de 6 meses,¿Cuándo te diagnosticaron?,Tengo VIH
1,1,1,2,Entre 6 meses y un año,¿Cuándo te diagnosticaron?,Tengo VIH
2,1,1,3,Hace más de un año,¿Cuándo te diagnosticaron?,Tengo VIH
3,1,2,4,Sí,¿Estás en tratamiento TAR?,Tengo VIH
4,1,2,5,No,¿Estás en tratamiento TAR?,Tengo VIH
5,1,2,6,No estoy seguro,¿Estás en tratamiento TAR?,Tengo VIH
6,1,3,7,Sí,¿Tienes acceso a un médico?,Tengo VIH
7,1,3,8,No,¿Tienes acceso a un médico?,Tengo VIH
8,2,4,9,Últimas 72h,¿Cuándo ocurrió la posible infección?,Creo que me he expuesto al virus
9,2,4,10,Hace más de 72h,¿Cuándo ocurrió la posible infección?,Creo que me he expuesto al virus


In [15]:
import json

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)


{
    "1.1.1": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace menos de 6 meses"
            ]
        }
    },
    "1.1.2": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Entre 6 meses y un año"
            ]
        }
    },
    "1.1.3": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Cuándo te diagnosticaron?": [
                "Hace más de un año"
            ]
        }
    },
    "1.1.4": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
                "Sí"
            ]
        }
    },
    "1.1.5": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
                "No"
            ]
        }
    },
    "1.1.6": {
        "titulo": "Tengo VIH",
        "preguntas": {
            "¿Estás en tratamiento TAR?": [
     

## probar base de datos a json

In [ ]:


except psycopg2.OperationalError as e:
    raise HTTPException(status_code=400, detail="Error de conexión: " + str(e))
except Exception as error:
    raise HTTPException(status_code=400, detail="Error desconocido: " + str(e))
    
# Escribe la consulta SQL
query = "SELECT * FROM sociosanitarios_data"

# Usa pandas para ejecutar la consulta y convertirla en un DataFrame
df = pd.read_sql_query(query, connection)

# Agrupar opciones por pregunta y categoría
grouped = df.groupby(["id_categoria", "titulo_categoria", "texto_pregunta"])["texto_opcion"].apply(list).reset_index()

# Crear el JSON dinámico
json_data = {}
for _, row in grouped.iterrows():
    id_categoria = row["id_categoria"]
    titulo = row["titulo_categoria"]
    pregunta = row["texto_pregunta"]
    opciones = row["texto_opcion"]
    
    if id_categoria not in json_data:
        json_data[id_categoria] = {"titulo": titulo, "preguntas": {}}
    
    json_data[id_categoria]["preguntas"][pregunta] = opciones

# Convertir a JSON
json_resultado = json.dumps(json_data, indent=4, ensure_ascii=False)

print(json_resultado)
